[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DoranLyong/NalCoding-For-DeepLearning/blob/main/codes/01_SLP_Regression/abalone_regression.ipynb)

In [9]:
import csv 
import time 

import numpy as np 

np.random.seed(1234)

# === Opt. === # 
RND_MEAN = 0
RND_STD = 0.0030
LEARNING_RATE = 0.001


# === 
data = None
input_cnt, output_cnt = None, None
weight, bias = None, None
shuffle_map, test_begin_idx = None, None


In [8]:
def load_abalone_dataset():
    with open('./data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
                
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

In [11]:
def init_model():
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [13]:
def arrange_data(mb_size):    
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)

    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size

    return step_count


def get_test_data():
    test_data = data[shuffle_map[test_begin_idx:]]

    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):

    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]

    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [14]:
def forward_neuralnet(x):
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

***
## Running Training 